In [1]:
import pandas as pd
import numpy as np
from indicnlp.tokenize import indic_tokenize
from transformers import AutoTokenizer
import torch

/home/dinesh/aiml_projects/language_translator/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
tokenizer = AutoTokenizer.from_pretrained("google-T5/T5-base")

In [5]:
data = pd.read_csv("Sentence pairs in English-Hindi - 2025-02-11.tsv",sep="\t",header=None,
                   names=["SrcSentenceID","SrcSentence","DstSentenceID","DstSentence"])

In [6]:
data.head()

,SrcSentenceID,SrcSentence,DstSentenceID,DstSentence
0,1282,Muiriel is 20 now.,485968,म्यूरियल अब बीस साल की हो गई है।
1,1282,Muiriel is 20 now.,2060319,म्यूरियल अब बीस साल की है।
2,1294,Education in this world disappoints me.,485564,मैं इस दुनिया में शिक्षा पर बहुत निराश हूँ।
3,1302,That won't happen.,2060320,वैसा नहीं होगा।
4,1308,I miss you.,2060321,मुझें तुम्हारी याद आ रही है।


In [7]:
data["DstSentence"] = data["DstSentence"].apply(lambda x: indic_tokenize.trivial_tokenize(x,lang="hi"))

In [8]:
data["SrcSentence"] = data["SrcSentence"].apply(tokenizer.tokenize)

In [9]:
data.head()

,SrcSentenceID,SrcSentence,DstSentenceID,DstSentence
0,1282,"[▁Mu, i, riel, ▁is, ▁20, ▁now, .]",485968,"[म्यूरियल, अब, बीस, साल, की, हो, गई, है, ।]"
1,1282,"[▁Mu, i, riel, ▁is, ▁20, ▁now, .]",2060319,"[म्यूरियल, अब, बीस, साल, की, है, ।]"
2,1294,"[▁Education, ▁in, ▁this, ▁world, ▁disappoint, ...",485564,"[मैं, इस, दुनिया, में, शिक्षा, पर, बहुत, निराश..."
3,1302,"[▁That, ▁won, ', t, ▁happen, .]",2060320,"[वैसा, नहीं, होगा, ।]"
4,1308,"[▁I, ▁miss, ▁you, .]",2060321,"[मुझें, तुम्हारी, याद, आ, रही, है, ।]"


In [10]:
data["SrcSentence"] = data["SrcSentence"].apply(tokenizer.convert_tokens_to_ids)

In [11]:
Vs = tokenizer.get_vocab()

In [12]:
Vd = set()
for tokenized_hindi_sentence in data["DstSentence"]:
    Vd.update(tokenized_hindi_sentence)

hindi_vocab = dict()
for idx, token in enumerate(Vd):
    hindi_vocab[token] = idx + 3

hindi_vocab["<PAD>"] = 0
hindi_vocab["<SOS>"] = 1
hindi_vocab["<EOS>"] = 2

Vd = hindi_vocab

In [13]:
data.head()

,SrcSentenceID,SrcSentence,DstSentenceID,DstSentence
0,1282,"[4159, 23, 14018, 19, 460, 230, 5]",485968,"[म्यूरियल, अब, बीस, साल, की, हो, गई, है, ।]"
1,1282,"[4159, 23, 14018, 19, 460, 230, 5]",2060319,"[म्यूरियल, अब, बीस, साल, की, है, ।]"
2,1294,"[2855, 16, 48, 296, 26963, 7, 140, 5]",485564,"[मैं, इस, दुनिया, में, शिक्षा, पर, बहुत, निराश..."
3,1302,"[466, 751, 31, 17, 1837, 5]",2060320,"[वैसा, नहीं, होगा, ।]"
4,1308,"[27, 3041, 25, 5]",2060321,"[मुझें, तुम्हारी, याद, आ, रही, है, ।]"


In [14]:
def convert_hindi_tokens_to_ids(tokenized_hindi_sentence):
    return [Vd[token] for token in tokenized_hindi_sentence]

In [15]:
data["DstSentence"] = data["DstSentence"].apply(convert_hindi_tokens_to_ids)

In [16]:
data.head()

,SrcSentenceID,SrcSentence,DstSentenceID,DstSentence
0,1282,"[4159, 23, 14018, 19, 460, 230, 5]",485968,"[56, 4099, 4351, 1270, 1865, 3586, 200, 3936, 67]"
1,1282,"[4159, 23, 14018, 19, 460, 230, 5]",2060319,"[56, 4099, 4351, 1270, 1865, 3936, 67]"
2,1294,"[2855, 16, 48, 296, 26963, 7, 140, 5]",485564,"[6501, 5698, 4889, 2736, 4794, 1162, 6788, 81,..."
3,1302,"[466, 751, 31, 17, 1837, 5]",2060320,"[5103, 911, 4246, 67]"
4,1308,"[27, 3041, 25, 5]",2060321,"[5486, 5262, 717, 5665, 5113, 3936, 67]"


In [17]:
def insert_sos_token_id(hindi_sentence_token_ids_list):
    return [1] + hindi_sentence_token_ids_list

In [18]:
def insert_eos_token_id(hindi_sentence_token_ids_list):
    return hindi_sentence_token_ids_list + [2]

In [19]:
data["DstSentenceInput"] = data["DstSentence"].apply(insert_sos_token_id)
data["DstSentenceLabel"] = data["DstSentence"].apply(insert_eos_token_id)

In [20]:
data.head()

,SrcSentenceID,SrcSentence,DstSentenceID,DstSentence,DstSentenceInput,DstSentenceLabel
0,1282,"[4159, 23, 14018, 19, 460, 230, 5]",485968,"[56, 4099, 4351, 1270, 1865, 3586, 200, 3936, 67]","[1, 56, 4099, 4351, 1270, 1865, 3586, 200, 393...","[56, 4099, 4351, 1270, 1865, 3586, 200, 3936, ..."
1,1282,"[4159, 23, 14018, 19, 460, 230, 5]",2060319,"[56, 4099, 4351, 1270, 1865, 3936, 67]","[1, 56, 4099, 4351, 1270, 1865, 3936, 67]","[56, 4099, 4351, 1270, 1865, 3936, 67, 2]"
2,1294,"[2855, 16, 48, 296, 26963, 7, 140, 5]",485564,"[6501, 5698, 4889, 2736, 4794, 1162, 6788, 81,...","[1, 6501, 5698, 4889, 2736, 4794, 1162, 6788, ...","[6501, 5698, 4889, 2736, 4794, 1162, 6788, 81,..."
3,1302,"[466, 751, 31, 17, 1837, 5]",2060320,"[5103, 911, 4246, 67]","[1, 5103, 911, 4246, 67]","[5103, 911, 4246, 67, 2]"
4,1308,"[27, 3041, 25, 5]",2060321,"[5486, 5262, 717, 5665, 5113, 3936, 67]","[1, 5486, 5262, 717, 5665, 5113, 3936, 67]","[5486, 5262, 717, 5665, 5113, 3936, 67, 2]"


In [21]:
data.drop(labels=["SrcSentenceID","DstSentenceID","DstSentence"],axis=1,inplace=True)

In [22]:
data.head()

,SrcSentence,DstSentenceInput,DstSentenceLabel
0,"[4159, 23, 14018, 19, 460, 230, 5]","[1, 56, 4099, 4351, 1270, 1865, 3586, 200, 393...","[56, 4099, 4351, 1270, 1865, 3586, 200, 3936, ..."
1,"[4159, 23, 14018, 19, 460, 230, 5]","[1, 56, 4099, 4351, 1270, 1865, 3936, 67]","[56, 4099, 4351, 1270, 1865, 3936, 67, 2]"
2,"[2855, 16, 48, 296, 26963, 7, 140, 5]","[1, 6501, 5698, 4889, 2736, 4794, 1162, 6788, ...","[6501, 5698, 4889, 2736, 4794, 1162, 6788, 81,..."
3,"[466, 751, 31, 17, 1837, 5]","[1, 5103, 911, 4246, 67]","[5103, 911, 4246, 67, 2]"
4,"[27, 3041, 25, 5]","[1, 5486, 5262, 717, 5665, 5113, 3936, 67]","[5486, 5262, 717, 5665, 5113, 3936, 67, 2]"


In [23]:
X = list(data["SrcSentence"])
Y_input = list(data["DstSentenceInput"])
Y_label = list(data["DstSentenceLabel"])

X_tensor = [torch.tensor(eng_tokenized_ids) for eng_tokenized_ids in X]
Y_input_tensor = [torch.tensor(hin_tokenized_ids) for hin_tokenized_ids in Y_input]
Y_label_tensor = [torch.tensor(hin_tokenized_ids) for hin_tokenized_ids in Y_label]

X_padded = torch.nn.utils.rnn.pad_sequence(X_tensor,batch_first=True)
Y_input_padded = torch.nn.utils.rnn.pad_sequence(Y_input_tensor,batch_first=True)
Y_label_padded = torch.nn.utils.rnn.pad_sequence(Y_label_tensor,batch_first=True)

In [24]:
Ns = X_padded.shape[1]
Nd = Y_label_padded.shape[1]

In [25]:
class Attention(torch.nn.Module):
    def __init__(self):
        super(Attention,self).__init__()
        self.attention_probabilities = torch.nn.Softmax(dim=1)

    def forward(self,encoder_outputs,decoder_lstm_layer_outputs):
        
        decoder_lstm_layer_outputs = torch.transpose(decoder_lstm_layer_outputs,dim0=1,dim1=2)
        alignment_scores = torch.bmm(encoder_outputs,decoder_lstm_layer_outputs)
        attention_weights = self.attention_probabilities(alignment_scores)
        attention_weights = torch.transpose(attention_weights,dim0=1,dim1=2)
        context_vectors = torch.bmm(attention_weights,encoder_outputs)

        return context_vectors

In [26]:
class Encoder(torch.nn.Module):
    def __init__(self,src_lang_vocab_size,topic_vector_dim):
        super(Encoder,self).__init__()
        self.first_emebdding_layer = torch.nn.Embedding(num_embeddings=src_lang_vocab_size,
                                                        embedding_dim=topic_vector_dim)
        self.second_lstm_layer = torch.nn.LSTM(input_size=topic_vector_dim,hidden_size=topic_vector_dim,
                                               batch_first=True)
        
    def forward(self,X_padded_mini_batch):

        first_embedding_layer_out = self.first_emebdding_layer(X_padded_mini_batch)
        encoder_outputs,(final_encoder_output,final_candidate_cell_state) = self.second_lstm_layer(first_embedding_layer_out)

        return encoder_outputs,(final_encoder_output,final_candidate_cell_state)

In [27]:
class Decoder(torch.nn.Module):
    def __init__(self,dst_lang_vocab_size,topic_vector_dim):
        super(Decoder,self).__init__()
        self.first_embedding_layer = torch.nn.Embedding(num_embeddings=dst_lang_vocab_size,
                                                        embedding_dim=topic_vector_dim)
        self.second_lstm_layer = torch.nn.LSTM(input_size=topic_vector_dim,hidden_size=topic_vector_dim,
                                               batch_first=True)
        self.attention_layer = Attention()
        self.output_layer = torch.nn.Linear(in_features=topic_vector_dim*2,out_features=dst_lang_vocab_size)
        self.output_layer_activation = torch.nn.Softmax(dim=2)

    def forward(self,encoder_outputs,initial_hidden_state,initial_candidate_cell_state,
                Y_padded_mini_batch):

        first_embedding_layer_out = self.first_embedding_layer(Y_padded_mini_batch)
        decoder_lstm_layer_outputs,final_cell_hidden_states = self.second_lstm_layer(first_embedding_layer_out,
                                                                                    (initial_hidden_state,
                                                                                    initial_candidate_cell_state))
        context_vectors = self.attention_layer(encoder_outputs,decoder_lstm_layer_outputs)
        concatenated_lstm_layer_output = torch.concatenate(tensors=(decoder_lstm_layer_outputs,context_vectors),dim=2)
        affine_transformed_output = self.output_layer(concatenated_lstm_layer_output)
        decoder_outputs = self.output_layer_activation(affine_transformed_output)

        return decoder_outputs

In [28]:
class Seq2SeqEncDecWithAttn(torch.nn.Module):
    def __init__(self,src_lang_vocab_size,dst_lang_vocab_size,topic_vector_dim):
        super(Seq2SeqEncDecWithAttn,self).__init__()
        self.encoder = Encoder(src_lang_vocab_size,topic_vector_dim)
        self.decoder = Decoder(dst_lang_vocab_size,topic_vector_dim)

    def forward(self,X_padded_mini_batch,Y_padded_mini_batch_input):

        encoder_outputs,(final_encoder_output,final_candidate_cell_state) = self.encoder(X_padded_mini_batch)
        Y_hat_mini_batch = self.decoder(encoder_outputs,final_encoder_output,final_candidate_cell_state,
                                       Y_padded_mini_batch_input)
        
        return Y_hat_mini_batch

In [29]:
X_padded_train = X_padded[0:13000]
Y_input_padded_train = Y_input_padded[0:13000]
Y_label_padded_train = Y_label_padded[0:13000]

X_padded_test = X_padded[13000:]
Y_input_padded_test = Y_input_padded[13000:]
Y_label_padded_test = Y_label_padded[13000:]

In [30]:
nw = Seq2SeqEncDecWithAttn(src_lang_vocab_size=len(Vs),dst_lang_vocab_size=len(Vd),topic_vector_dim=32)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(params=nw.parameters())
epochs = 5
mb_size = 26

for epoch in range(epochs):
    for i in range(X_padded_train.shape[0]//mb_size):

        X_train_mb = X_padded_train[i*mb_size:(i+1)*mb_size]
        Y_input_train_mb = Y_input_padded_train[i*mb_size:(i+1)*mb_size]
        Y_label_train_mb = Y_label_padded_train[i*mb_size:(i+1)*mb_size]

        Y_label_train_mb = Y_label_train_mb.reshape(Y_label_train_mb.shape[0]*Y_label_train_mb.shape[1],)

        Y_pred_train_mb = nw(X_train_mb,Y_input_train_mb)
        Y_pred_train_mb = Y_pred_train_mb.reshape(Y_pred_train_mb.shape[0]*Y_pred_train_mb.shape[1],
                                                  Y_pred_train_mb.shape[2])
        

        loss_fn_value = loss_fn(Y_pred_train_mb,Y_label_train_mb)

        loss_fn_value.backward()
        optimizer.step()
        optimizer.zero_grad()

        if i % 25 == 0:
            print("Epoch #{}, Mini Batch #{}, CCE Loss = {}".format(epoch,i,loss_fn_value))

Epoch #0, Mini Batch #0, CCE Loss = 8.863907814025879
Epoch #0, Mini Batch #25, CCE Loss = 7.995378017425537
Epoch #0, Mini Batch #50, CCE Loss = 7.98037052154541
Epoch #0, Mini Batch #75, CCE Loss = 7.9785075187683105
Epoch #0, Mini Batch #100, CCE Loss = 7.994322776794434
Epoch #0, Mini Batch #125, CCE Loss = 7.977442741394043
Epoch #0, Mini Batch #150, CCE Loss = 7.962618350982666
Epoch #0, Mini Batch #175, CCE Loss = 7.974462032318115
Epoch #0, Mini Batch #200, CCE Loss = 7.950715065002441
Epoch #0, Mini Batch #225, CCE Loss = 7.955216884613037
Epoch #0, Mini Batch #250, CCE Loss = 7.958611011505127
Epoch #0, Mini Batch #275, CCE Loss = 7.956344127655029
Epoch #0, Mini Batch #300, CCE Loss = 7.9382429122924805
Epoch #0, Mini Batch #325, CCE Loss = 7.991974353790283
Epoch #0, Mini Batch #350, CCE Loss = 7.956912517547607
Epoch #0, Mini Batch #375, CCE Loss = 7.947290420532227
Epoch #0, Mini Batch #400, CCE Loss = 7.960298538208008
Epoch #0, Mini Batch #425, CCE Loss = 7.961995124816